**ອິມພັອດດ ໄລບາລີ້**
------

In [ ]:
!pip install bidict

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from bidict import bidict
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

**ດາວໂຫຼດ ດາຕ້າ**

In [ ]:
labels = np.load('data_2/labels.npy')
images= np.load('data_2/images.npy')

In [ ]:
ENCODER = bidict({
    'ກ':1,'ຂ':2,'ຄ':3,'ງ':4,'ຈ':5,'ສ':6,'ຊ':7,'ຍ':8,'ດ':9,'ຕ':10,'ຖ':11,'ທ':12,
    'ນ':13,'ບ':14,'ປ':15,'ຜ':16,'ຝ':17,'ພ':18,'ຟ':19,'ມ':20,'ຢ':21,'ຣ':22,'ລ':23,'ວ':24,
    'ຫ':25,'ອ':26,'ຮ':27
})

**ປ່ຽນ ເລໂບ້ຈາກ ອັກສອນ ເປັນ ຕົວເລກ**

In [ ]:
labels = np.array([ENCODER[x] for x in labels])

In [ ]:
labels

In [ ]:
imgs = images.astype("float32") / 255
imgs.shape

ພັລອດ ອອກມາສະແດງພາບຂໍ້ມູນ

In [ ]:
plt.figure()
plt.imshow(imgs[5])
plt.grid(False)
plt.show()

ເພີ່ມ ດິມເມຊັ່ນໃຫ້ image

In [ ]:
imgs = np.expand_dims(imgs, -1)

In [ ]:
imgs.shape

**ແບ່ງຂໍ້ມູນ ອອກເປັນ ເທລນ ແລະ ເທັດ**

In [ ]:
labels, imgs = shuffle(labels, imgs)
split = .75

labels_train = labels[:int(len(labels) * split)]
labels_test = labels[int(len(labels) * split):]

imgs_train = imgs[:int(len(imgs) * split)]
imgs_test = imgs[int(len(imgs) * split):]

In [ ]:
# ເບິ່ງໂຄງສ້າງຂອງຂໍ້ມູນ 
imgs_train.shape

In [ ]:
batch_size = 16
epochs = 20

model = keras.Sequential([
    keras.Input(shape=(50, 50, 1)),
    layers.Conv2D(32, kernel_size=3, activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.Dropout(0.22),
    layers.Conv2D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.Dropout(0.22),
  
    layers.Flatten(),
    layers.Dense(len(ENCODER)+1, activation='softmax')
])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2)
optimizer = keras.optimizers.Adam()

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

**ຝິດ ໂມເດວວ**

In [ ]:
model.fit(imgs_train,
          labels_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(imgs_test, labels_test),
          callbacks=[early_stopping])

**ທົດສອບບໂມເດວ**

In [ ]:
labels_pred = np.argmax(model.predict(imgs_test), axis=-1)

In [ ]:
cm = confusion_matrix(labels_test, labels_pred, labels=list(ENCODER.inverse.keys()))

In [ ]:
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, cbar=False, cmap="Blues", xticklabels=list(ENCODER.keys()), yticklabels=list(ENCODER.keys()))
plt.show()